Thanks for @christofhenkel @abhishek @iezepov for their great work:

https://www.kaggle.com/christofhenkel/how-to-preprocessing-for-glove-part2-usage
https://www.kaggle.com/abhishek/pytorch-bert-inference
https://www.kaggle.com/iezepov/starter-gensim-word-embeddings

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import sys
package_dir = "../input/russian-bert-pavlov/"
sys.path.append(package_dir)

In [ ]:
#%reload_ext autoreload
#%autoreload 
#%matplotlib inline

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
from tqdm import tqdm
import warnings
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig
from nltk.tokenize.treebank import TreebankWordTokenizer
from scipy.stats import rankdata

from gensim.models import KeyedVectors
#import fastai
#from fastai.train import Learner
#from fastai.train import DataBunch
#from fastai.callbacks import *
#from fastai.basic_data import DatasetType
#import fastprogress
#from fastprogress import force_console_behavior
import numpy as np
from pprint import pprint
import pandas as pd
import os
import time
import gc
import random
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing import text, sequence
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F





In [ ]:
SEED =1234

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
import torch
warnings.filterwarnings(action='once')
device = torch.device('cuda')
MAX_SEQUENCE_LENGTH = 256
SEED = 1234
BATCH_SIZE = 512
BERT_MODEL_PATH = '../input/russian-bert-pavlov/'
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
bert_config = BertConfig('../input/berttinkoff/bert_config.json')
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

tqdm.pandas()
CRAWL_EMBEDDING_PATH = '../input/fasttest-common-crawl-russian/cc.ru.300.vec'
#GLOVE_EMBEDDING_PATH = '../input/gensim-embeddings-dataset/glove.840B.300d.gensim'
NUM_MODELS = 1
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 200
if not is_interactive():
    def nop(it, *a, **k):
        return it

    tqdm = nop

    fastprogress.fastprogress.NO_BAR = True
    master_bar, progress_bar = force_console_behavior()
    fastai.basic_train.master_bar, fastai.basic_train.progress_bar = master_bar, progress_bar

seed_everything()

**BERT Part**

In [ ]:
Data_dir = "../input/datatinkoff/"

d = 473141
num_to_load= 373141  #

valid_size= 100000

In [ ]:
def merge_text(x):
    
    
    return "{0} . {1} . {2}".format(x['header'], x['text'], x['job_title']) 


train_df  =  pd.read_csv(os.path.join(Data_dir,"sr_train.csv")).sample(d,random_state=SEED)


train_df.loc[train_df.job_title.isnull(), 'job_title']  = 'Пусто'

train_df.loc[train_df.header.isnull(), 'header']  = 'Пусто'

train_df = train_df.tail(100000)


train_df['text_1'] = train_df[['header', 'text', 'job_title']].apply(lambda x: merge_text(x), axis=1)
train_df['text_1'] = train_df['text_1'].progress_apply(lambda x:str(x))
X_train = convert_lines(train_df["text_1"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)


#sequences = convert_lines(train_df["text_1"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)


#test_df = pd.read_csv(os.path.join(Data_dir,"sr_test.csv"))
#train_df['comment_text'] = train_df['comment_text'].astype(str) 



test_df  =  pd.read_csv(os.path.join(Data_dir,"sr_test.csv"))#.sample(d,random_state=SEED)


test_df.loc[test_df.job_title.isnull(), 'job_title']  = 'Пусто'

test_df.loc[test_df.header.isnull(), 'header']  = 'Пусто'



test_df['text_1'] = test_df[['header', 'text', 'job_title']].apply(lambda x: merge_text(x), axis=1)
test_df['text_1'] = test_df['text_1'].apply(lambda x:str(x))
X_test = convert_lines(test_df["text_1"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)



##test_df['comment_text'] = test_df['comment_text'].astype(str) 
#X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

In [ ]:
model = BertForSequenceClassification(bert_config, num_labels=1)
model.load_state_dict(torch.load("../input/berttinkoff/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.requires_grad = False
model.eval()

In [ ]:
test_preds = np.zeros((len(X_test)  ,  4))
test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    #test_preds[i * 512:(i + 1) * 512] = pred[:, 0].detach().cpu().squeeze().numpy()
    test_preds[i * 512:(i + 1) * 512] = pred[:, 0].detach().cpu().squeeze().numpy()
    test_preds[i * 512:(i + 1) * 512] = pred[:, 1].detach().cpu().squeeze().numpy()
    test_preds[i * 512:(i + 1) * 512] = pred[:, 2].detach().cpu().squeeze().numpy()
    test_preds[i * 512:(i + 1) * 512] = pred[:, 3].detach().cpu().squeeze().numpy()
    

#test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()


MODEL_NAME = ['like', 'skip', 'dislike',  'view']
for i in range(len(MODEL_NAME)):
    test_df[MODEL_NAME[i]]=torch.sigmoid(torch.tensor(test_preds[:,i])).numpy()

In [ ]:
#submission_bert_test = pd.DataFrame.from_dict({
#    'id': test_df['id'],
##    'prediction': test_pred
#})
test_df.to_csv("bertestk.csv")

In [ ]:
train_preds = np.zeros((len(X_train)  ,  4))
train = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.long))
train_loader = torch.utils.data.DataLoader(train, batch_size=512, shuffle=False)
tk0 = tqdm(train_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    #train_preds[i * 512:(i + 1) * 512] = pred[:, 0].detach().cpu().squeeze().numpy()
    
    train_preds[i * 512:(i + 1) * 512,0]=pred[:,0].detach().cpu().squeeze().numpy()
    train_preds[i * 512:(i + 1) * 512,1]=pred[:,1].detach().cpu().squeeze().numpy()
    train_preds[i * 512:(i + 1) * 512,2]=pred[:,2].detach().cpu().squeeze().numpy()
    train_preds[i * 512:(i + 1) * 512,3]=pred[:,3].detach().cpu().squeeze().numpy()
    

#train_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

MODEL_NAME = ['like', 'skip', 'dislike',  'view']
for i in range(len(MODEL_NAME)):
    train_df[MODEL_NAME[i]]=torch.sigmoid(torch.tensor(valid_preds[:,i])).numpy()
    


In [ ]:

train_df.to_csv("berttrain100000k.csv")

**LSTM Part**

In [ ]:
##train_df = reduce_mem_usage(pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv'))



In [ ]:
##symbols_to_isolate = '.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊɒ∞§{}·τα❤☺ɡ|¢→̶`❥━┣┫┗Ｏ►★©―ɪ✔®\x96\x92●£♥➤´¹☕≈÷♡◐║▬′ɔː€۩۞†μ✒➥═☆ˌ◄½ʻπδηλσερνʃ✬ＳＵＰＥＲＩＴ☻±♍µº¾✓◾؟．⬅℅»Вав❣⋅¿¬♫ＣＭβ█▓▒░⇒⭐›¡₂₃❧▰▔◞▀▂▃▄▅▆▇↙γ̄″☹➡«φ⅓„✋：¥̲̅́∙‛◇✏▷❓❗¶˚˙）сиʿ✨。ɑ\x80◕！％¯−ﬂﬁ₁²ʌ¼⁴⁄₄⌠♭✘╪▶☭✭♪☔☠♂☃☎✈✌✰❆☙○‣⚓年∎ℒ▪▙☏⅛ｃａｓǀ℮¸ｗ‚∼‖ℳ❄←☼⋆ʒ⊂、⅔¨͡๏⚾⚽Φ×θ￦？（℃⏩☮⚠月✊❌⭕▸■⇌☐☑⚡☄ǫ╭∩╮，例＞ʕɐ̣Δ₀✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒↑☝ɹ✅☛♩☞ＡＪＢ◔◡↓♀⬆̱ℏ\x91⠀ˤ╚↺⇤∏✾◦♬³の｜／∵∴√Ω¤☜▲↳▫‿⬇✧ｏｖｍ－２０８＇‰≤∕ˆ⚜☁'
##symbols_to_delete = '\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xad😢🐶️\uf0e0😜😎👊\u200b\u200e😁عدويهصقأناخلىبمغر😍💖💵Е👎😀😂\u202a\u202c🔥😄🏻💥ᴍʏʀᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ😋👏שלוםבי😱‼\x81エンジ故障\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘אעכח💩💯⛽🚄🏼ஜ😖ᴠ🚲‐😟😈💪🙏🎯🌹😇💔😡\x7f👌ἐὶήιὲκἀίῃἴξ🙄Ｈ😠\ufeff\u2028😉😤⛺🙂\u3000تحكسة👮💙فزط😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷улкнПоАН🐾🐕😆ה🔗🚽歌舞伎🙈😴🏿🤗🇺🇸мυтѕ⤵🏆🎃😩\u200a🌠🐟💫💰💎эпрд\x95🖐🙅⛲🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢᵒʳʸᴼᴷᴺʷᵗʰᵉᵘ\x13🚬🤓\ue602😵άοόςέὸתמדףנרךצט😒͝🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10成都😣⏺😌🤑🌏😯ех😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613小土豆🏡❔⁉\u202f👠》कर्मा🇹🇼🌸蔡英文🌞🎲レクサス😛外国人关系Сб💋💀🎄💜🤢َِьыгя不是\x9c\x9d🗑\u2005💃📣👿༼つ༽😰ḷЗз▱ц￼🤣卖温哥华议会下降你失去所有的钱加拿大坏税骗子🐝ツ🎅\x85🍺آإشء🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003🚀🤴ʲшчИОРФДЯМюж😝🖑ὐύύ特殊作戦群щ💨圆明园קℐ🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪天一家⚲\u2006⚭⚆⬭⬯⏖新✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺ϖ\u2000үսᴦᎥһͺ\u2007հ\u2001ɩｙｅ൦ｌƽｈ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧Ƅᴨןᑯ໐ΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓ё𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘ͦ💸ج패티Ｗ𝙇ᵻ👂👃ɜ🎫\uf0a7БУі🚢🚂ગુજરાતીῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼☘﴾̯﴿₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸公寓养宠物吗🏄🐀🚑🤷操美𝒑𝒚𝒐𝑴🤙🐒欢迎来到阿拉斯ספ𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅ﷻ🦄巨收赢得白鬼愤怒要买额ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟斤拷𝗢𝟳𝟱𝟬⦁マルハニチロ株式社⛷한국어ㄸㅓ니͜ʖ𝘿𝙔₵𝒩ℯ𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅ℴ𝒻𝒽𝓀𝓌𝒸𝓎𝙏ζ𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊多伦🐽🎻🎹⛓🏹🍷🦆为和中友谊祝贺与其想象对法如直接问用自己猜本传教士没积唯认识基督徒曾经让相信耶稣复活死怪他但当们聊些政治题时候战胜因圣把全堂结婚孩恐惧且栗谓这样还♾🎸🤕🤒⛑🎁批判检讨🏝🦁🙋😶쥐스탱트뤼도석유가격인상이경제황을렵게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용🔫👁凸ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡知彼百\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳点击查版🍭𝑥𝑦𝑧ＮＧ👣\uf020っ🏉ф💭🎥Ξ🐴👨🤳🦍\x0b🍩𝑯𝒒😗𝟐🏂👳🍗🕉🐲چی𝑮𝗕𝗴🍒ꜥⲣⲏ🐑⏰鉄リ事件ї💊「」\uf203\uf09a\uf222\ue608\uf202\uf099\uf469\ue607\uf410\ue600燻製シ虚偽屁理屈Г𝑩𝑰𝒀𝑺🌤𝗳𝗜𝗙𝗦𝗧🍊ὺἈἡχῖΛ⤏🇳𝒙ψՁմեռայինրւդձ冬至ὀ𝒁🔹🤚🍎𝑷🐂💅𝘬𝘱𝘸𝘷𝘐𝘭𝘓𝘖𝘹𝘲𝘫کΒώ💢ΜΟΝΑΕ🇱♲𝝈↴💒⊘Ȼ🚴🖕🖤🥘📍👈➕🚫🎨🌑🐻𝐎𝐍𝐊𝑭🤖🎎😼🕷ｇｒｎｔｉｄｕｆｂｋ𝟰🇴🇭🇻🇲𝗞𝗭𝗘𝗤👼📉🍟🍦🌈🔭《🐊🐍\uf10aლڡ🐦\U0001f92f\U0001f92a🐡💳ἱ🙇𝗸𝗟𝗠𝗷🥜さようなら🔼'

In [ ]:
##tokenizer = TreebankWordTokenizer()

##isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
##remove_dict = {ord(c):f'' for c in symbols_to_delete}

In [ ]:
##x_train = train_df['text_1'].progress_apply(lambda x:preprocess(x))
##y_aux_train = train_df[['like', 'skip','dislike', 'view']]
##x_test = test_df['text_1'].progress_apply(lambda x:preprocess(x))
##col = ['like', 'skip','dislike', 'view']
##identity_columns = [
 #   'like', 'skip','dislike', 'view']
# Overall
##weights = np.ones((len(x_train),)) / 4
# Subgroup
##weights += (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
##weights += (( (train_df[col].values>=0.5).astype(bool).astype(np.int) +
 ##  (train_df[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
# Background Negative, Subgroup Positive
##weights += (( (train_df[col].values<0.5).astype(bool).astype(np.int) +
 #3  (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
##loss_weight = 1.0 / weights.mean()

##y_train = np.vstack([(train_df[col].values>=0.5).astype(np.int),weights]).T

##max_features = 410047

In [ ]:
###tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False)

In [ ]:
###tokenizer.fit_on_texts(list(x_train) + list(x_test))

###crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
####print('n unknown words (crawl): ', len(unknown_words_crawl))

#glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
#print('n unknown words (glove): ', len(unknown_words_glove))

####max_features = max_features or len(tokenizer.word_index) + 1
####max_features

####embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
###embedding_matrix.shape

###del crawl_matrix
#del glove_matrix
####gc.collect()

####y_train_torch = torch.tensor(np.hstack([y_train, y_aux_train]), dtype=torch.float32)

In [ ]:
##x_train = tokenizer.texts_to_sequences(x_train)
##x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
###lengths = torch.from_numpy(np.array([len(x) for x in x_train]))
 
###maxlen = 300
###x_train_padded = torch.from_numpy(sequence.pad_sequences(x_train, maxlen=maxlen))

In [ ]:
###test_lengths = torch.from_numpy(np.array([len(x) for x in x_test]))

###x_test_padded = torch.from_numpy(sequence.pad_sequences(x_test, maxlen=maxlen))

In [ ]:
###batch_size = 512
###test_dataset = data.TensorDataset(x_test_padded, test_lengths)
###train_dataset = data.TensorDataset(x_train_padded, lengths, y_train_torch)
###valid_dataset = data.Subset(train_dataset, indices=[0, 1])

###train_collator = SequenceBucketCollator(lambda lenghts: lenghts.max(), 
   ###                                     sequence_index=0, 
      ###                                  length_index=1, 
         ##                               label_index=2)
####test_collator = SequenceBucketCollator(lambda lenghts: lenghts.max(), sequence_index=0, length_index=1)

####train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_collator)
###valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=train_collator)
###test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_collator)

###databunch = DataBunch(train_dl=train_loader, valid_dl=valid_loader, collate_fn=train_collator)

In [ ]:
###all_test_preds = []

##for model_idx in range(NUM_MODELS):
 ##   print('Model ', model_idx)
  ##  seed_everything(1 + model_idx)
  ##  model = NeuralNet(embedding_matrix, y_aux_train.shape[-1])
  # # learn = Learner(databunch, model, loss_func=custom_loss)
  ##  test_preds = train_model(learn,test_dataset,output_dim=7)    
    ##all_test_preds.append(test_preds)

In [ ]:
#submission_lstm = pd.DataFrame.from_dict({#
#    'id': test_df['id'],
#    'prediction': np.mean(all_test_preds, axis=0)[:, 0]
#})



###MODEL_NAME = ['like', 'skip', 'dislike',  'view']
##for i in range(len(MODEL_NAME)):
##    train_df[MODEL_NAME[i]]=torch.sigmoid(torch.tensor(valid_preds[:,i])).numpy()

**Blending part**

In [ ]:
#submission = pd.read_csv(#
 #   "../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv"
#)

#weights = [0.333, 0.667]
#weights = [0.5, 0.5]
#submission["prediction"] = ensemble_predictions(
#    [submission_bert.prediction.values, submission_lstm.prediction.values],
##    weights,
 #   type_="rank",
#)
#submission.to_csv("submission.csv", index=False)


